In [1]:
# The goal is to train an embedding with a simple dataset where 
# the word relationships are very clear.

In [2]:
import re
import string
import pandas as pd
import tensorflow as tf
keras = tf.keras

from model import Word2Vec

from tensorflow.keras.optimizers import Adam

In [7]:
# Read json from data/easy_sentences.json
df = pd.read_json('data/easy_sentences.json')
df.rename(columns={0: 'description'}, inplace=True)
df

,description
0,bulbasaur ivysaur venusaur grass poison
1,charmander charmeleon charizard fire flying
2,squirtle wartortle blastoise water
3,caterpie metapod butterfree bug flying
4,weedle kakuna beedrill bug poison
5,pidgey pidgeotto pidgeot normal flying
6,rattata raticate normal
7,spearow fearow normal flying
8,ekans arbok poison
9,pikachu raichu electric


In [8]:
# Put together a training corpus
descriptions = list(df["description"].values)

# Clean and tokenize text
def preprocess_text(text):
    text = text.lower()  # lowercase
    text = re.sub(f"[{string.punctuation}]", "", text)  # remove punctuation
    words = text.split()  # tokenize into words
    return words

# Apply to all sentences
dataset = [preprocess_text(sentence) for sentence in descriptions]
print(dataset)

[['bulbasaur', 'ivysaur', 'venusaur', 'grass', 'poison'], ['charmander', 'charmeleon', 'charizard', 'fire', 'flying'], ['squirtle', 'wartortle', 'blastoise', 'water'], ['caterpie', 'metapod', 'butterfree', 'bug', 'flying'], ['weedle', 'kakuna', 'beedrill', 'bug', 'poison'], ['pidgey', 'pidgeotto', 'pidgeot', 'normal', 'flying'], ['rattata', 'raticate', 'normal'], ['spearow', 'fearow', 'normal', 'flying'], ['ekans', 'arbok', 'poison'], ['pikachu', 'raichu', 'electric'], ['sandshrew', 'sandslash', 'ground'], ['nidoran', 'nidorina', 'nidoqueen', 'poison', 'ground'], ['nidoran', 'nidorino', 'nidoking', 'poison', 'ground'], ['clefairy', 'clefable', 'fairy'], ['vulpix', 'ninetales', 'fire'], ['jigglypuff', 'wigglytuff', 'normal', 'fairy'], ['zubat', 'golbat', 'poison', 'flying'], ['oddish', 'gloom', 'vileplume', 'grass', 'poison'], ['paras', 'parasect', 'bug', 'grass'], ['venonat', 'venomoth', 'bug', 'poison'], ['diglett', 'dugtrio', 'ground'], ['meowth', 'persian', 'normal'], ['psyduck', 'g

In [31]:
window_size = 12
embedding_dim = 5

epochs = 100
# Use Adam optimizer with learning rate decay
optimizer = tf.keras.optimizers.legacy.Adam(
    learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-3,
        decay_steps=1000,
        decay_rate=0.96
    )
)

# better with neg examples
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
#loss = tf.keras.losses.CosineSimilarity()

word2vec = Word2Vec(dataset, embedding_dim)
train_dataset = word2vec.prepare_dataset(window_size)
word2vec.compile(optimizer, loss, metrics=['accuracy'])
word2vec.fit(train_dataset, epochs=epochs)

Sample word pairs: [('bulbasaur', 'ivysaur'), ('bulbasaur', 'venusaur'), ('bulbasaur', 'grass'), ('bulbasaur', 'poison'), ('ivysaur', 'bulbasaur')]
Epoch 1/100
10/10 [==============================] - 1s 27ms/step - loss: 0.7038 - accuracy: 0.5457
Epoch 2/100
10/10 [==============================] - 0s 9ms/step - loss: 0.6678 - accuracy: 0.5834
Epoch 3/100
10/10 [==============================] - 0s 9ms/step - loss: 0.6405 - accuracy: 0.6329
Epoch 4/100
10/10 [==============================] - 0s 8ms/step - loss: 0.6157 - accuracy: 0.6706
Epoch 5/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5954 - accuracy: 0.7033
Epoch 6/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5778 - accuracy: 0.7485
Epoch 7/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5640 - accuracy: 0.7678
Epoch 8/100
10/10 [==============================] - 0s 6ms/step - loss: 0.5526 - accuracy: 0.7821
Epoch 9/100
10/10 [==============================] - 0s 7ms

In [32]:
word2vec.summary()

Model: "word2_vec_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 word_embedding (Embedding)  multiple                  635       
                                                                 
 dot_5 (Dot)                 multiple                  0         
                                                                 
Total params: 635 (2.48 KB)
Trainable params: 635 (2.48 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
# Example usage
word2vec.get_word_embedding("pikachu")

array([ 0.04023632, -0.01369237,  0.05323527, -0.01768201, -0.04289438],
      dtype=float32)

In [34]:
# Example usage
word2vec.compute_similarity("bulbasaur", "grass")

0.7020279

In [35]:
# Example usage
word2vec.compute_similarity("bulbasaur", "fire")

-0.41641998

In [17]:
from evaluation_tests.poke_types import POKEMONS_BY_TYPE

all_pokemons = []
for pokemons in POKEMONS_BY_TYPE.values():
    all_pokemons.extend(pokemons)

print(all_pokemons)

['charmander', 'charmeleon', 'charizard', 'vulpix', 'ninetales', 'growlithe', 'arcanine', 'ponyta', 'rapidash', 'magmar', 'flareon', 'moltres', 'squirtle', 'wartortle', 'blastoise', 'psyduck', 'golduck', 'poliwag', 'poliwhirl', 'poliwrath', 'tentacool', 'tentacruel', 'slowpoke', 'slowbro', 'seel', 'dewgong', 'shellder', 'cloyster', 'krabby', 'kingler', 'horsea', 'seadra', 'goldeen', 'seaking', 'staryu', 'starmie', 'magikarp', 'gyarados', 'lapras', 'bulbasaur', 'ivysaur', 'venusaur', 'oddish', 'vileplume', 'paras', 'parasect', 'bellsprout', 'weepinbell', 'victreebel', 'exeggcute', 'exeggutor', 'tangela']


In [20]:
pokemon_to_type = {}
for type_name, pokemon_list in POKEMONS_BY_TYPE.items():
    for pokemon in pokemon_list:
        pokemon_to_type[pokemon] = type_name

poke_types = [pokemon_to_type[poke] for poke in all_pokemons]
poke_and_types = {poke: poke_type for poke, poke_type in zip(all_pokemons, poke_types)}

In [21]:
pokes_to_remove = []
for pokemon in poke_and_types.keys():
    if not (pokemon in word2vec.vocabulary):
        pokes_to_remove.append(pokemon)         

for poke in pokes_to_remove:
    poke_and_types.pop(poke)

In [22]:
len(all_pokemons)

52

In [43]:
from model_utils import visualize_embeddings

visualize_embeddings(word2vec, dim=2, words=poke_and_types, rnd_seed=123)

In [13]:

# Save the model to a directory
model_save_path = "./saved_model"
word2vec.save(model_save_path)

INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets
